In [148]:
import networkx as nx
import pandas as pd
import json

from matplotlib import pyplot as plt
from collections.abc import Mapping
from typing import Any, Literal
from itertools import product
from pathlib import Path

In [149]:
def load_records(result_file_name: str) -> list[Mapping[str, Any]]:
    records: list[Mapping[str, Any]] = []
    with open(Path("./results") / result_file_name, 'r') as file:
        dpth: int = 0
        buff: list[str] = []
        for raw in file:
            line = raw.strip()
            if not line:
                continue
            dpth += line.count('{')
            dpth -= line.count('}')
            buff.append(raw)
            if dpth == 0 and buff:
                chunk = ''.join(buff)
                obj = json.loads(chunk)
                records.append(obj)
                buff = []
    return records

In [150]:
def best_record(records: list[Mapping[str, Any]]) -> Mapping[str, Any]:
    best: Mapping[str, Any] = {}
    for record in records:
        for uid, details in record.items():
            if not best or details['score'] < best['score']:
                best = {'id': uid, **details}
    return best

In [151]:
def load_config(result_file_name: str) -> Mapping[str, Any]:
    stem = Path(result_file_name).stem            
    config_name = stem.removeprefix("result_")      
    
    config_path = Path("configurations") / f"{config_name}.json"
    
    # 3) Open & load the JSON
    with config_path.open("r", encoding="utf-8") as f:
        config = json.load(f)
    return config

In [152]:
def load_experiment(experiment_name: str) -> Mapping[str, Any]:
    result_file_name = f"result_{experiment_name}.jsonl"
    records = load_records(result_file_name)
    best = best_record(records)
    config = load_config(result_file_name)
    return {
        "name": experiment_name,
        "result": best,
        "config": config
    }

In [153]:
def load_all_experiments() -> list[Mapping[str, Any]]:
    experiments: list[Mapping[str, Any]] = []
    for result_file in Path("./results").glob("result_*.jsonl"):
        experiment_name = result_file.stem.removeprefix("result_")
        experiment = load_experiment(experiment_name)
        experiments.append(experiment)
    return experiments

In [154]:
def only_experiments_with(algorithm: Literal["ST", "TF", "GDFS"], weights: tuple[float, float, float, float]) -> list[Mapping[str, Any]]:
    experiments = load_all_experiments()
    filtered: list[Mapping[str, Any]] = []
    for experiment in experiments:
        config = experiment["config"]
        optimizer = config["optimizer"]
        if config["reconstruction_algo"] == algorithm and optimizer["weights"] == list(weights):
            filtered.append(experiment)
    return filtered

In [155]:
def extract_label(experiment: Mapping[str, Any]) -> str:
    solver = experiment["config"]["solverName"]
    algorithm = experiment["config"]["reconstruction_algo"]
    hyperparams = experiment["config"]["optimizer"]
    hyperparams = {k: v for k, v in hyperparams.items() if k not in ["max_iterations", "objective", "nested"]}
    proto_label = {
        "solver": solver,
        "algorithm": algorithm,
        "hyperparams": hyperparams
    }
    return json.dumps(proto_label, sort_keys=True)

In [156]:
def dominance_network(experiments: list[Mapping[str, Any]]) -> nx.Graph:
    network = nx.DiGraph()
    for exp_a, exp_b in product(experiments, repeat=2):
        if exp_a == exp_b:
            continue
        score_a = exp_a["result"]["score"]
        score_b = exp_b["result"]["score"]
        if score_a < score_b:
            network.add_edge(extract_label(exp_a), extract_label(exp_b))
    return network

In [157]:
def plot_dominance_network(network: nx.DiGraph) -> None:
    """
    Plot the dominance network where an edge from A to B means A has a better (lower) score than B.
    
    Args:
        network: A NetworkX DiGraph representing the dominance relationships
    """
    
    # Create figure
    plt.figure(figsize=(14, 10))
    
    # Use a layout that works well for directed graphs
    pos = nx.spring_layout(network, seed=42)
    
    # Create shorter labels for readability
    labels = {}
    for node in network.nodes():
        try:
            data = json.loads(node)
            solver = data['solver']
            algo = data['algorithm']
            params = data['hyperparams']
            # Create a short representation that captures key info
            short_label = f"{solver[:3]}-{algo}\n{params.get('learning_rate', '')}"
            labels[node] = short_label
        except:
            labels[node] = node[:15] + "..."
    
    # Draw the network
    nx.draw_networkx_nodes(network, pos, node_size=700, node_color='skyblue', alpha=0.8)
    nx.draw_networkx_edges(network, pos, width=1.0, alpha=0.7, 
                          arrowsize=15, edge_color='gray')
    nx.draw_networkx_labels(network, pos, labels=labels, font_size=9)
    
    # Add a title and legend
    plt.title("Experiment Dominance Network\n(Arrow points from better to worse configuration)")
    plt.axis("off")
    
    # Show plot
    plt.tight_layout()
    plt.show()

In [158]:
def rank_nodes_by_dominance(network: nx.DiGraph) -> list[tuple[str, int, float]]:
    """
    Rank nodes in the dominance network by the number of configurations they dominate.
    
    Args:
        network: A NetworkX DiGraph representing dominance relationships
        
    Returns:
        A list of (node_label, outgoing_edges_count, dominance_ratio) sorted by count in descending order
    """
    # Get the out-degree (number of outgoing edges) for each node
    out_degrees = dict(network.out_degree())
    
    # Get total number of nodes (excluding self)
    total_possible_dominance = len(network.nodes()) - 1
    
    # Calculate dominance ratio and create the ranking list
    ranked_nodes = []
    for node, count in out_degrees.items():
        # Dominance ratio: what percentage of other configs this one dominates
        dominance_ratio = count / max(1, total_possible_dominance)
        ranked_nodes.append((node, count, dominance_ratio))
    
    # Sort by count in descending order
    ranked_nodes.sort(key=lambda x: x[1], reverse=True)
    
    return ranked_nodes

In [159]:
def display_dominance_ranking(ranked_nodes: list[tuple[str, int, float]], network: nx.DiGraph) -> None:
    """
    Display the ranked nodes with their dominance metrics in a readable format.
    
    Args:
        ranked_nodes: List of (node_label, outgoing_edges_count, dominance_ratio)
        network: The dominance network to extract in-degree information
    """
    
    # Get the in-degree (number of incoming edges) for each node
    in_degrees = dict(network.in_degree())
    
    # Create a table for display
    print(f"{'Rank':<5} {'Dominates':<10} {'Dominated by':<12} {'Ratio':<8} {'Configuration':<45}")
    print("-" * 90)
    
    for rank, (node, count, ratio) in enumerate(ranked_nodes, 1):
        # Get number of configurations that dominate this one
        dominated_by = in_degrees.get(node, 0)
        
        try:
            data = json.loads(node)
            solver = data['solver']
            algo = data['algorithm']
            params = data['hyperparams']
            
            # Create a readable configuration string
            key_params = {k: v for k, v in params.items() 
                         if k in ['learning_rate', 'momentum', 'beta']}
            config_str = data
        except:
            config_str = node[:42] + "..."
            
        print(f"{rank:<5} {count:<10} {dominated_by:<12} {ratio:.2f}     {config_str}")

In [160]:
all_algorithms: list[str] = ["ST", "TF"]
all_weights: list[tuple[float, float, float, float]] = [
    (0.1, 0.3, 0.5, -10.0),
    (0.01, 0.5, 0.5, -7.0),
    (0.4, 0.2, 0.5, -7.0),
    (0.4, 0.2, 0.3, -5.0),
]
for algorithm, weights in product(all_algorithms, all_weights):
    print(f"Algorithm: {algorithm}, Weights: {weights}")
    filtered_experiments = only_experiments_with(algorithm, weights)
    if not filtered_experiments:
        print("No experiments found for this configuration.")
        continue
    network = dominance_network(filtered_experiments)
    display_dominance_ranking(rank_nodes_by_dominance(network), network)
    print("\n" + "="*80 + "\n")

Algorithm: ST, Weights: (0.1, 0.3, 0.5, -10.0)
Rank  Dominates  Dominated by Ratio    Configuration                                
------------------------------------------------------------------------------------------
1     15         0            1.00     {'algorithm': 'ST', 'hyperparams': {'resolution': 7, 'weights': [0.1, 0.3, 0.5, -10.0]}, 'solver': 'GridSearch'}
2     14         1            0.93     {'algorithm': 'ST', 'hyperparams': {'resolution': 10, 'weights': [0.1, 0.3, 0.5, -10.0]}, 'solver': 'GridSearch'}
3     13         2            0.87     {'algorithm': 'ST', 'hyperparams': {'learning_rate': 0.001, 'target_score': 0.1, 'weights': [0.1, 0.3, 0.5, -10.0]}, 'solver': 'Bayesian'}
4     12         3            0.80     {'algorithm': 'ST', 'hyperparams': {'learning_rate': 0.01, 'target_score': 0.3, 'weights': [0.1, 0.3, 0.5, -10.0]}, 'solver': 'Bayesian'}
5     11         4            0.73     {'algorithm': 'ST', 'hyperparams': {'learning_rate': 0.1, 'target_score': 0.3,